In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [37]:
import json
import math

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk
from nltk.corpus import stopwords

In [9]:
path = 'recipes_with_calories_protein_fat_and_sodium.csv'

In [10]:
df = pd.read_csv(path, sep="|", index_col=0)

In [11]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

In [12]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.000000,7.000000,559.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.000000,23.000000,1439.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.000000,7.000000,165.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,252.445801,14.630608,23.051666,0.662995,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.000000,32.000000,452.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [14]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [15]:
def tokenize(word_list):
    return word_list.split('|')

In [17]:
foods = []
for i in range(len(df)):
    foods.append(get_small_dict(df, keep_columns, i))

In [18]:
possible_tags = set()
for my_dict in foods:
    possible_tags |= set(my_dict['tags'])

In [19]:
le = LabelEncoder()
le.fit(list(possible_tags))

LabelEncoder()

In [20]:
# 'crêpe' in df.columns
# df[df['spritzer'] == 1]

In [21]:
drink_tags = {'alcoholic', 'amaretto', 'apple juice', 'bitters', 'bourbon', 'brandy',
              'campari', 'champagne', 'chartreuse', 'cocktail', 'cocktail party', 'coffee',
              'cognac/armagnac', 'digestif', 'drink', 'drinks', 'eau de vie', 'egg nog',
              'fortified wine', 'frangelico', 'fruit juice', 'gin', 'grand marnier',
              'hot drink', 'house cocktail', 'iced coffee', 'iced tea', 'juicer', 'kahlua',
              'kirsch', 'liqueur', 'margarita', 'martini', 'non-alcoholic', 'orange juice',
              'red wine', 'rum', 'sparkling wine', 'spritzer', 'tea', 'tequila', 'vermouth',
              'vodka', 'whiskey', 'white wine', 'wine'}

In [22]:
dessert_tags = {'butterscotch/caramel', 'candy', 'cake', 'chocolate', 'cookie', 'cookies',
                'creme de cacao', 'crepe', 'crme de cacao', 'cupcake', 'dessert', 'double boiler',
                'ice cream', 'ice cream machine'}

In [24]:
df[df.breakfast == 1].shape[0], df[df.lunch == 1].shape[0], df[df.dinner == 1].shape[0]

(712, 1359, 2705)

In [25]:
df[df.side == 1].shape[0], df[df.snack == 1].shape[0]

(3151, 28)

In [26]:
le.transform(['kahlua', 'bon appetit', 'breakfast', 'lunch', 'dinner', 'snack'])

array([300,  57,  66, 350, 180, 568])

In [290]:
from itertools import permutations
from itertools import combinations

NUM_PLANS = 3

POPULATION_SIZE = 60
SELECTION_SIZE = 14
CROSSOVER_SIZE = POPULATION_SIZE - SELECTION_SIZE

MUTATION_FACTOR = .4
NUMBER_OF_GENERATIONS = 200
SP = 2

In [28]:
from random import choice

In [ ]:
delete_titles = ['To Sterilize Jars and Lids', 'Roquefort Mayonnaise']

In [35]:
mapping = [
    'breakfast',
    'snack1',
    'lunch',
    'snack2',
    'dinner',
    'dessert',
    'drink'
]

In [44]:
EPS = 1e-1
INF = 10000

In [95]:
import math
from random import sample
from random import random

In [225]:
def evaluate_plan(foods, solution, mapping, calories_target, preferences=[], verbose=False):
    
    solution_full_data = [
        {
            'title': foods[v]['title'],
            'calories': foods[v]['calories'],
            'type': mapping[i],
            'protein': foods[v]['protein'],
            'fat': foods[v]['fat'],
            'sodium': foods[v]['sodium'],
            'tags': foods[v]['tags']
        }
            for i, (k, v) in enumerate(solution.items())
    ]
    assert len(solution_full_data) > 0, ValueError("Can't process empty solution! Aborting.")
    
    calories_sum = np.sum([s['calories'] for s in solution_full_data[:-2]])
    calories_target = calories_target
    calories_diff = abs(calories_sum - calories_target)
    if verbose:
        print("current diff", calories_diff)
    
    evaluation = .1 * calories_diff
    
    for preference in preferences:
        target = preference['target']
        restrictor = preference['restrictor']
        values = preference['values']
        
        if target in solution_full_data[0].keys():
            if restrictor == "<":
                real_sum = np.sum([s[target] for s in solution_full_data])
                diff = abs(real_sum - values)
                if verbose:
                    print("current diff - < ", diff)
                if real_sum < values:
                    evaluation -= .1 * diff
                else:
                    evaluation += .1 * diff
                    
            elif restrictor == "<=":
                real_sum = np.sum([s[target] for s in solution_full_data])
                diff = abs(real_sum - values)
                if verbose:
                    print("current diff - <= ", diff)
                if real_sum <= values:
                    evaluation -= .1 * diff
                else:
                    evaluation += .1 * diff
            elif restrictor == ">":
                real_sum = np.sum([s[target] for s in solution_full_data])
                diff = abs(real_sum - values)
                if verbose:
                    print("current diff - > ", diff)
                if real_sum > values:
                    evaluation -= .1 * diff
                else:
                    evaluation += .1 * diff
            elif restrictor == ">=":
                real_sum = np.sum([s[target] for s in solution_full_data])
                diff = abs(real_sum - values)
                if verbose:
                    print("current diff - >= ", diff)
                if real_sum >= values:
                    evaluation -= .1 * diff
                else:
                    evaluation += .1 * diff
            elif restrictor == "isin":
                assert target == 'tags', ValueError("The <isin> operator should be used only with <tags> data!")
                assert isinstance(values, list), ValueError("The <values> operator should be a list!")

                real_tags = [s[target] for s in solution_full_data]
                real_tags = [item for sublist in real_tags for item in sublist]
                diff = len(set(real_tags).intersection(set(values)))
                if verbose:
                    print("current diff - isin ", diff)
                evaluation -= .1 * diff
                
            elif restrictor == "notin":
                assert target == 'tags', ValueError("The <notin> operator should be used only with <tags> data!")
                assert isinstance(values, list), ValueError("The <values> operator should be a list!")

                real_tags = [s[target] for s in solution_full_data]
                real_tags = [item for sublist in real_tags for item in sublist]
                diff = len(set(real_tags).intersection(set(values)))
                if verbose:
                    print("current diff - notin ", diff)
                evaluation += .1 * diff
    plan = [
        [
            {'title': foods[v]['title'], 'calories': foods[v]['calories'], 'type': mapping[i]}
                for i, (k, v) in enumerate(solution.items())
        ]
    ]

    return evaluation, plan

In [226]:
def generate_random_individual(domains, foods, mapping, calories_target, preferences):

    solution = { 
        i: choice(domains[i]) 
            for i in range(len(domains))
    }
    
    while True:
        evaluation, plan = evaluate_plan(
            foods,
            solution,
            mapping,
            calories_target,
            preferences=preferences
        )
        
        solution = { 
            i: choice(domains[i]) 
                for i in range(len(domains))
        }

        if evaluation != -INF:
            break

    return solution

In [228]:
def mutate_individual(individual, domains):
    for variable in individual:
        if random() < MUTATION_FACTOR:
            individual[variable] = choice(domains[variable])
    return individual

In [229]:
def crossover(ind1, ind2, genes):
    crossover_point = choice(range(len(genes) - 1))
    child1, child2 = {}, {}
    for i, gene in enumerate(genes):
        if i < crossover_point:
            child1[gene] = ind1[gene]
            child2[gene] = ind2[gene]
        else:
            child1[gene] = ind2[gene]
            child2[gene] = ind1[gene]

    return child1, child2


In [230]:
def scale_fitnesses(fitnesses):
    sortind = np.argsort(fitnesses).tolist()
    return [2 - SP + 2 * (SP - 1) * ((sortind.index(i) + 1) - 1) / (len(fitnesses) - 1)
            for i in range(len(fitnesses))]

In [263]:
def compute_fitnesses(population, return_plans=False):
    fitnesses = []
    plans = []
    for individual in population:
        evaluation, plan = evaluate_plan(
            foods,
            individual,
            mapping,
            calories_target=CALORIES_TARGET,
            preferences=preferences
        )
        fitnesses.append(-evaluation)
        plans.append(plan)
    fitnesses = scale_fitnesses(fitnesses)
    return fitnesses if not return_plans else (fitnesses, plans)


In [302]:
preferences_headache = [
    {
        'target': 'sodium',
        'restrictor': '<=',
        'values': 2300
    },
    {
        'target': 'tags',
        'restrictor': 'isin',
        'values': [
            'salmon',
            'sardine',
            'fish',
            'avocado',
        ]
    },
    {
        'target': 'tags',
        'restrictor': 'notin',
        'values': [
            'cheese'
        ]
    }
]

In [306]:
preferences_diabetes = [
    {
        'target': 'sodium',
        'restrictor': '<=',
        'values': 1500
    },
    {
        'target': 'fat',
        'restrictor': '<',
        'values': 3000
    },
    {
        'target': 'tags',
        'restrictor': 'isin',
        'values': [
            'lentil',
            'fruit',
        ]
    },
    {
        'target': 'tags',
        'restrictor': 'isin',
        'values': [
            'salmon',
            'sardine',
            'fish',
            'avocado',
        ]
    },
    {
        'target': 'tags',
        'restrictor': 'notin',
        'values': [
            'beef',
            'bacon',
            'sausage'
        ]
    }
]

In [350]:
CALORIES_TARGET=3000

In [351]:
l = [f['tags'] for f in foods]
l = [item for sublist in l for item in sublist]

In [352]:
'salmon' in set(l)

True

In [353]:
domain_breakfast = []
domain_snack = []
domain_lunch = []
domain_dinner = []
domain_dessert = []
domain_drinks = []
for i, food in enumerate(foods):
    if 'breakfast' in food['tags'] and food['calories'] <= CALORIES_TARGET/2:
        domain_breakfast.append(i)
    if 'snack' in food['tags'] and food['calories'] <= CALORIES_TARGET/2:
        domain_snack.append(i)
    if 'lunch' in food['tags'] and food['calories'] <= CALORIES_TARGET/2:
        domain_lunch.append(i)
    if 'dinner' in food['tags'] and food['calories'] <= CALORIES_TARGET/2:
        domain_dinner.append(i)
    if len(dessert_tags.intersection(set(food['tags']))) > 1 and \
        i not in domain_breakfast and \
        i not in domain_snack and \
        i not in domain_lunch and \
        i not in domain_dinner and \
        food['calories'] <= CALORIES_TARGET/2:
            domain_dessert.append(i)
    elif len(drink_tags.intersection(set(food['tags']))) > 1 and \
        i not in domain_breakfast and \
        i not in domain_snack and \
        i not in domain_lunch and \
        i not in domain_dinner and \
        food['calories'] <= CALORIES_TARGET/2:
            domain_drinks.append(i)
             

domains = [
    domain_breakfast,
    domain_snack,
    domain_lunch,
    domain_snack,
    domain_dinner,
    domain_dessert,
    domain_drinks
]

In [354]:
[len(d) for d in domains], sum([len(d) for d in domains])

([702, 28, 1328, 28, 2552, 1501, 1202], 7341)

In [367]:
print('Generating initial population')
population = [
    generate_random_individual(
        domains,
        foods,
        mapping,
        calories_target=CALORIES_TARGET,
        preferences=preferences_diabetes
    ) for _ in range(POPULATION_SIZE)
]
print('Done generating initial population')

Generating initial population
Done generating initial population


In [368]:
evaluation, plan = evaluate_plan(foods, solution, mapping, CALORIES_TARGET, preferences=preferences_diabetes)

In [369]:
evaluation, plan

(10666.000000000002,
 [[{'calories': 438.0,
    'title': 'Mozzarella and Tomato Sauce Omelets',
    'type': 'breakfast'},
   {'calories': 267.0,
    'title': '3-Ingredient Cinnamon-Sugar Twists',
    'type': 'snack1'},
   {'calories': 222.0,
    'title': 'Cured Salmon with Fennel and Carrot Salad',
    'type': 'lunch'},
   {'calories': 208.0, 'title': 'Quark (Sour Fresh Cheese)', 'type': 'snack2'},
   {'calories': 553.0,
    'title': 'Whole Fish Baked in Sea Salt',
    'type': 'dinner'},
   {'calories': 114.0,
    'title': 'Oatmeal Cookies with Chocolate Chunks and Candied Ginger',
    'type': 'dessert'},
   {'calories': 466.0, 'title': 'Mussels au Gratin', 'type': 'drink'}]])

In [370]:
NUMBER_OF_GENERATIONS = 1000

In [371]:
for generation in range(NUMBER_OF_GENERATIONS):
    fitnesses = compute_fitnesses(population)
    best_individuals = np.argsort(fitnesses)
    new_population = [population[e] for e in best_individuals[:SELECTION_SIZE]]
    children = []
    for _ in range(int(CROSSOVER_SIZE / 2)):
        child1, child2 = crossover(
            *sample(new_population, 2),
            genes=[i for i in range(len(domains))]
        )
        child1 = mutate_individual(child1, domains=domains)
        child2 = mutate_individual(child2, domains=domains)
        children.extend([child1, child2])
    new_population.extend(children)

    if len(new_population) < POPULATION_SIZE:
        new_population += sample(population, POPULATION_SIZE - len(new_population))
    elif len(new_population) > POPULATION_SIZE:
        new_population = new_population[:POPULATION_SIZE]

    population = new_population

# Get the best individuals
fitnesses, plans = compute_fitnesses(population, return_plans=True)    
best_individual = np.argmin(fitnesses)

In [372]:
existing_data = []
for p in population:
    existing_data.append('_'.join([str(i) for i in p.values()]))

In [373]:
existing_data = set(existing_data)

In [374]:
reduced_population = []
for p in population:
    if '_'.join([str(i) for i in p.values()]) in existing_data:
        reduced_population.append(p)

In [375]:
final_meal_plans = []
for _ in range(SELECTION_SIZE):
    final_meal_plans.append(choice(reduced_population))

In [376]:
pretty_plans = []
for solution in final_meal_plans:
    plan = [
        {'title': foods[v]['title'], 'calories': foods[v]['calories'], 'type': mapping[i]}
            for i, (k, v) in enumerate(solution.items())
    ]
    pretty_plans.append(plan)

In [377]:
print(json.dumps(pretty_plans, indent=4))

[
    [
        {
            "type": "breakfast",
            "title": "Beet Cured Lox",
            "calories": 324.0
        },
        {
            "type": "snack1",
            "title": "Banana Bread",
            "calories": 274.0
        },
        {
            "type": "lunch",
            "title": "Cured Salmon with Fennel and Carrot Salad",
            "calories": 222.0
        },
        {
            "type": "snack2",
            "title": "Quark (Sour Fresh Cheese)",
            "calories": 208.0
        },
        {
            "type": "dinner",
            "title": "Moroccan Slow-Cooked Lamb",
            "calories": 777.0
        },
        {
            "type": "dessert",
            "title": "Raspberry-Chocolate Tart",
            "calories": 474.0
        },
        {
            "type": "drink",
            "title": "Frozen Sunrise Margaritas",
            "calories": 282.0
        }
    ],
    [
        {
            "type": "breakfast",
            "title": "Beet 

In [332]:
solution = population[best_individual]

In [319]:
plan = [
    {'title': foods[v]['title'], 'calories': foods[v]['calories'], 'type': mapping[i]}
        for i, (k, v) in enumerate(solution.items())
]

In [322]:
[p['calories'] for p in plan], sum([p['calories'] for p in plan])

([324.0, 208.0, 222.0, 208.0, 553.0, 194.0, 282.0], 1991.0)

In [ ]:
population